<a href="https://colab.research.google.com/github/puc26/myProject/blob/main/AttendanceCalculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 出勤規則：
1. 標準上班時間：08:30 - 18:00
2. 彈性上班時間：09:00 - 18:30
3. 午休時間：12:00 - 13:30
4. 超過上班時間半小時後開始計算加班，以每半小時為單位計算
5. 程式返回值：總出勤時間、遲到時間、早退時間、請假時間、加班時間

### 使用說明：
資料格式：直接使用數字，開頭為零請捨去。（ex. 08:30 -> 830, 17:30 -> 1730）  
start_time：上班打卡時間  
end_time：下班打卡時間  
leave_times：請假時間，可多個時段。（ex. leave_times = ((830, 1000), (1600, 1800))）

# 定義演算式

In [ ]:
import datetime
import pandas as pd

In [ ]:
# 合併上下班時段及請假時段，排除午休時段，返回上午及下午總時段
def merge_times(start_time, end_time, leave_times):
    # 定義時間格式
    format_str = '%H%M'

    merged_times = []

    # 將上班時間範圍加入合併後的列表
    merged_times.append((start_time, end_time))

    # 將請假時間範圍加入合併後的列表
    for leave_start, leave_end in leave_times:
        merged_times.append((leave_start, leave_end))

    # 根據時間的起始時間進行排序
    merged_times.sort(key=lambda x: x[0])

    # 午休時段
    noon_start = '1200'
    noon_end = '1330'

    # 合併重疊的時間範圍
    merged_result = []
    current_start = None
    current_end = None

    for start_time, end_time in merged_times:
        if current_start is None:
            current_start = start_time
            current_end = end_time
        elif start_time <= current_end:
            current_end = max(current_end, end_time)
        else:
            merged_result.append((current_start, current_end))
            current_start = start_time
            current_end = end_time

    if current_start is not None:
        merged_result.append((current_start, current_end))

    # 分割上午和下午的合併時段
    morning_result = []
    afternoon_result = []
    vacancy = []

    for start_time, end_time in merged_result:
        if end_time <= noon_start:
            morning_result.append((start_time, end_time))
        elif start_time >= noon_end:
            afternoon_result.append((start_time, end_time))
        else:
            if start_time < noon_start:
                morning_result.append((start_time, noon_start))
            if end_time > noon_end:
                afternoon_result.append((noon_end, end_time))

    # 加入空缺時段
    for i in range(len(morning_result) - 1):
      end_time = morning_result[i][-1]
      start_time = morning_result[i+1][0]
      if end_time != start_time:
        vacancy.append((end_time, start_time))

    for i in range(len(afternoon_result) - 1):
      end_time = afternoon_result[i][-1]
      start_time = afternoon_result[i+1][0]
      if end_time != start_time:
        vacancy.append((end_time, start_time))

    # 加入鄰近午休時間的空缺時段
    if len(morning_result) > 0 and morning_result[-1][1] != '1200':
      vacancy.append((morning_result[-1][1], '1200'))
    if len(afternoon_result) > 0 and afternoon_result[0][0] != '1330':
      vacancy.append(('1330', afternoon_result[0][0]))

    # 加入未上班時段
    if len(morning_result) == 0:
      vacancy.append(('0830', '1200'))
    if len(afternoon_result) == 0:
      vacancy.append(('1330', '1800'))

    return morning_result, afternoon_result, vacancy


In [ ]:
# 定義出勤計算機
def work_times(start_time, end_time, leave_times):
  # 轉換使用者輸入資料格式
  start_time = str(start_time).zfill(4)
  end_time = str(end_time).zfill(4)
  leave_times = [(str(start_time).zfill(4), str(end_time).zfill(4)) for start_time, end_time in leave_times]

  # 定義時間格式
  format_str = '%H%M'

  # 初始化參數
  late = 0
  early_leave = 0
  overtime = 0
  total_leave = 0

  # 正常上班時間範圍
  morning_work_start = datetime.datetime.strptime('0830', format_str)
  morning_work_end = datetime.datetime.strptime('1200', format_str)
  afternoon_work_start = datetime.datetime.strptime('1330', format_str)
  afternoon_work_end = datetime.datetime.strptime('1800', format_str)
  # 彈性上下班時間的範圍
  flex_morning_work_start = datetime.datetime.strptime('0900', format_str)
  flex_afternoon_work_end = datetime.datetime.strptime('1830', format_str)

  # 檢查是否有空缺時段
  morning_result, afternoon_result, vacancy = merge_times(start_time, end_time, leave_times)
  if len(vacancy) > 0:
    return '上班時段有空缺時段，請檢查時段： ', vacancy, ' !!'

  # 上下午時段皆存在
  if len(morning_result) != 0 and len(afternoon_result) != 0:
    start = datetime.datetime.strptime(morning_result[0][0], format_str)
    end = datetime.datetime.strptime(afternoon_result[-1][-1], format_str)

    # 計算遲到與早退
    if start <= morning_work_start: # 早於上班時間，起始時間等於正常上班時間
      start = morning_work_start
    elif morning_work_start < start <= flex_morning_work_start: # 開始上班於彈性上班範圍內
      afternoon_work_end += (start - morning_work_start) # 重新正常下班時間
    else: # 開始上班時間超過彈性上班時間
      afternoon_work_end = flex_afternoon_work_end

    late = (start - flex_morning_work_start).total_seconds() // 60 if start > flex_morning_work_start else 0
    early_leave = (afternoon_work_end - end).total_seconds() // 60 if end < afternoon_work_end else 0

  # 上午時段存在、下午時段不存在
  elif len(morning_result) != 0 and len(afternoon_result) == 0:
    start = datetime.datetime.strptime(morning_result[0][0], format_str)
    end = datetime.datetime.strptime(morning_result[-1][-1], format_str)

    # 計算遲到與早退
    if start <= morning_work_start: # 開始上班小於等於正常上班時間
      start = morning_work_start
    elif morning_work_start < start <= flex_morning_work_start: # 開始上班於彈性上班範圍內
      afternoon_work_end += (start - morning_work_start) # 重新定義正常下班時間
    else: # 開始上班時間超過彈性上班時間
      afternoon_work_end = flex_afternoon_work_end # 重新定義彈性下班時間

    late = (start - flex_morning_work_start).total_seconds() // 60 if start > flex_morning_work_start else 0
    early_leave = (afternoon_work_end - end).total_seconds() // 60 if end < afternoon_work_end else 0

  # 上午時段不存在、下午時段存在
  elif len(morning_result) == 0 and len(afternoon_result) != 0:
    start = datetime.datetime.strptime(afternoon_result[0][0], format_str)
    end = datetime.datetime.strptime(afternoon_result[-1][-1], format_str)

    # 計算遲到與早退
    if start <= afternoon_work_end: # 開始上班小於午休時間
      start = afternoon_work_end
    else: # 開始上班大於午休時間
      afternoon_work_end = flex_afternoon_work_end # 重新定義正常下班時間

    late = (start - flex_morning_work_start).total_seconds() // 60 - 90
    early_leave = (afternoon_work_end - end).total_seconds() // 60

  # 上下午時段皆不存在
  else:
      return '不存在上班時段，請確認 !!'

  # 加班小節統計
  overtime_count = (end - afternoon_work_end).total_seconds() // (datetime.timedelta(minutes=30)).total_seconds() - 1
  # 檢查融合時間下班是否晚於加班時間
  if end >= afternoon_work_end:
    if overtime_count > 0:
      overtime = overtime_count * 30 # 晚於加班時間，達到加班標準，計算加班時間
      end = afternoon_work_end
    else:
      end = afternoon_work_end # 晚於下班時間，未達加班標準
  else:
    early_leave = (afternoon_work_end - end).total_seconds() // 60 # 早於下班時間，計算早退時間

  # 轉換leave_times為datetime.datetime
  leave_times = [(datetime.datetime.strptime(start_time, format_str), datetime.datetime.strptime(end_time, format_str)) for start_time, end_time in leave_times]
  # 計算請假時間
  for leave_start, leave_end in leave_times:
    leave_duration = leave_end - leave_start
    if leave_start < morning_work_end and leave_end > afternoon_work_start:
      leave_duration -= datetime.timedelta(minutes=90)
    total_leave += leave_duration.total_seconds() // 60

  # 計算上班分鐘數
  total_min = (end-start).total_seconds() // 60
  total_min -= total_leave
  if start < morning_work_end and end > afternoon_work_start:
    total_min -= 90

  # 轉換為DataFrame格式
  result = {'分鐘數': [total_min, late, early_leave, total_leave, overtime],
            '時數': [total_min/60, late/60, early_leave/60, total_leave/60, overtime/60]}
  df = pd.DataFrame(result).round(2)
  df.index = ['總時間', '遲到', '早退', '請假', '加班']

  return df

# 計算出勤時間

In [ ]:
# 參數設定
start_time = 830
end_time = 1700
leave_times = [(1700, 1800)]

# 代入演算式（不需更改）
work_times(start_time, end_time, leave_times)

,分鐘數,時數
總時間,420.0,7.0
遲到,0.0,0.0
早退,0.0,0.0
請假,60.0,1.0
加班,0.0,0.0


# Author
This notebook is created by Steven Chen.  
*Jul 9 2023*  
v1.0